In [1]:
import nltk
import spacy
from collections import Counter
from database.pymysql_conn import DataBase
from tqdm import tqdm

from gensim.parsing.preprocessing import remove_stopwords

import pickle

tqdm.pandas()

C:\ProgramData\Anaconda3\envs\pytorch-gpu-V\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
db = DataBase()

In [3]:
SQL="""
SELECT 
    T1.appid,
    T1.avg_player_count,
    T1.gameName,
    T1.release_date,
    T2.publishedAt,
    T2.text,    
    DATEDIFF(T2.publishedAt, T1.release_date) as datediff
FROM
    (SELECT 
        A.appid, A.gameName, A.avg_player_count, B.release_date
    FROM
        (SELECT 
        *
    FROM
        yt.games) A
    JOIN (SELECT 
        appid, name, MAX(release_date) AS release_date
    FROM
        oasis.app_info2
    GROUP BY appid) AS B ON A.appid = B.appid) T1
        LEFT JOIN
    (SELECT 
        appid, gameName, text, publishedAt
    FROM
        steam.yt_comment
    WHERE
        filter = 0 AND language = 'en') T2 ON T1.appid = T2.appid
WHERE
    DATEDIFF(T2.publishedAt, T1.release_date) <= 300
"""

In [14]:
df = db.to_df(SQL)

In [15]:
spacy_en = spacy.load("en")

def tokenize(text):
    # text = remove_stopwords(text)
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [36]:
case1 = df['text'][1]

In [37]:
case1=  nltk.word_tokenize(case1)

In [38]:
case2 = tokenize(df['text'][1])

In [39]:
nltk.tag.pos_tag(case2)

[('I', 'PRP'),
 ('agree', 'VBP'),
 (',', ','),
 ('in', 'IN'),
 ('doaescarlet', 'NN'),
 ('you', 'PRP'),
 ('can', 'MD'),
 ('collect', 'VB'),
 ('money', 'NN'),
 ('for', 'IN'),
 ('costumes', 'NNS'),
 ('by', 'IN'),
 ('playing', 'VBG'),
 ('(', '('),
 ('o', 'JJ'),
 ('r', 'NN'),
 (' ', 'NNP'),
 ('buying', 'VBG'),
 ('tickets', 'NNS'),
 ('with', 'IN'),
 ('real', 'JJ'),
 ('money', 'NN'),
 (')', ')'),
 ('but', 'CC'),
 ('at', 'IN'),
 ('least', 'JJS'),
 ('you', 'PRP'),
 ('can', 'MD'),
 ('play', 'VB'),
 ('only', 'RB'),
 ('and', 'CC'),
 ('do', 'VBP'),
 ("n't", 'RB'),
 ('spend', 'VB'),
 ('I', 'PRP'),
 ('have', 'VBP'),
 ('to', 'TO'),
 ('say', 'VB'),
 ('that', 'IN'),
 ('doa', 'NN'),
 ('6', 'CD'),
 ('has', 'VBZ'),
 ('scandalous', 'JJ'),
 ('dlc', 'NN'),
 ('prices', 'NNS'),
 ('but', 'CC'),
 ('the', 'DT'),
 ('core', 'NN'),
 ('game', 'NN'),
 ('costs', 'VBZ'),
 ('less', 'JJR'),
 ('I', 'PRP'),
 ('do', 'VBP'),
 ("n't", 'RB'),
 ('understand', 'VB'),
 ('why', 'WRB'),
 ('tecmo', 'JJ'),
 ('koei', 'NN'),
 ('makes', '

In [ ]:
lines = line
# function to test if something is a noun
is_noun = lambda pos: 'NN' in pos[:2]
is_adj = lambda pos: pos[:2] == 'JJ'
# do the nlp stuff
tokenized = nltk.word_tokenize(lines)
nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
adjectives = [word for (word, pos) in nltk.pos_tag(tokenized) if is_adj(pos)]

In [21]:
is_noun = lambda pos: pos[:2] == 'NN'
is_adj = lambda pos: pos[:2] == 'JJ'

def pos_tagging_noun(text):
    ###########
    tokenized = tokenize(text)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
    return nouns

def pos_tagging_adj(text):
    tokenized = nltk.word_tokenize(text)
    adjectives = [word for (word, pos) in nltk.pos_tag(tokenized) if is_adj(pos)]
    return adjectives

In [22]:
def get_noun_count(df):
    
    nouns = df['text'].progress_apply(pos_tagging_noun)
    unpacked = [word for sent in nouns for word in sent]
    
    noun_cnt = Counter(unpacked)
    
    return noun_cnt

def get_adj_count(df):
    
    adjs = df['text'].progress_apply(pos_tagging_adj)
    unpacked = [word for sent in adjs for word in sent]
    
    adj_cnt = Counter(unpacked)
    
    return adj_cnt

In [23]:
noun_cnt = get_noun_count(df[:100])

100%|██████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 476.27it/s]


In [ ]:
adj_cnt = get_adj_count(df)

In [ ]:
noun_cnt.most_common()

In [ ]:
adj_cnt.most_common()

In [ ]:
# with open('noun.pickle', 'wb') as f:
#     pickle.dump(noun_cnt, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('adj.pickle', 'wb') as f:
#     pickle.dump(adj_cnt, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('noun.pickle', 'rb') as f:
#     data = pickle.load(f)